# Delete Cycle

This notebook permanently deletes active cycle from your Risk Modeling workflow. 

1. Deletes the _Active directory 
2. Removes the cycle, stages and steps from the database

**Note:** This action cannot be reversed!

In [5]:
import shutil
import sys
from helpers import cycle, database as db, ux, constants
from helpers.context import WorkContext
from helpers.step import Step

## Look into current cycles (if any)

In [2]:
import os
os.getcwd()

'/home/jovyan/workspace/workflows/_Tools/Cycle Management'

In [8]:
ux.subheader("Current Cycles")

cycles_df = cycle.get_cycle_status()
if not cycles_df.empty:
    ux.dataframe(cycles_df, max_rows=10)
else:
    ux.info("No cycles found")

# Check for active cycle
active_cycle = cycle.get_active_cycle()
if active_cycle:
    if ux.yes_no(f"Do you want to delete {active_cycle['cycle_name']}?"):
        shutil.rmtree(constants.WORKFLOWS_PATH / f"Active_{active_cycle['cycle_name']}")
        db.delete_cycle((active_cycle['id'],))
    else:
        ux.error("Execution stopped.")
        exit
else:
    ux.info("No active cycle")

id,cycle_name,status,created_ts,archived_ts
4,Analysis-2025-Q2-v4,ACTIVE,2025-10-20 03:20:45.660191+00:00,NaT
3,Analysis-2025-Q2-v3,ARCHIVED,2025-10-20 03:16:18.610099+00:00,2025-10-20 03:20:45.648453+00:00
2,Analysis-2025-Q2-v2,ARCHIVED,2025-10-20 03:08:33.300002+00:00,2025-10-20 03:16:18.597217+00:00
1,Analysis-2025-Q2,ARCHIVED,2025-10-20 03:04:18.291231+00:00,2025-10-20 03:08:33.284032+00:00


Do you want to delete Analysis-2025-Q2-v4? (y/n):  n


## Cleanup Active_ directori(es)

In [11]:
# Check filesystem
from pathlib import Path
workflows_path = Path('/home/jovyan/workspace/workflows')
active_dirs = [d for d in workflows_path.iterdir() if d.is_dir() and d.name.startswith('Active_')]
if active_dirs:
    ux.warning(f"Active directory found: {', '.join(d.name for d in active_dirs)}")
    for active_dir in active_dirs:
        if ux.yes_no(f"Do you want to delete {active_dir.name}?"):
            shutil.rmtree(constants.WORKFLOWS_PATH / active_dir.name) 
            ux.success(f"Deleted {constants.WORKFLOWS_PATH / active_dir.name}") 
        else:
            ux.warning(f"Retaining {constants.WORKFLOWS_PATH / active_dir.name}") 
            continue
else:
    ux.info("No active directory found")

Do you want to delete Active_Analysis-2025-Q2-v4? (y/n):  y


Do you want to delete Active_Analysis-2020-Q1-twiceagain? (y/n):  y


---